In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta
from pprint import pprint

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.helper_functions import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design
from demres.demins.functions import *

In [4]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [5]:
pt_features

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,...,fgas_pdds_window0,fgas_pdds_window1,sgas_pdds_window0,sgas_pdds_window1,sga_depots_pdds_window0,sga_depots_pdds_window1,fga_depots_pdds_window0,fga_depots_pdds_window1,antidepressants_pdds_window0,antidepressants_pdds_window1
0,10665377,1,1920,377,2011-04-07,True,4693.0,2001-04-01,2011-04-14,55943.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
1,12440330,2,1925,330,2010-03-30,True,4693.0,2000-03-22,2010-04-09,50163.0,...,30.0,0.00,0.0,490.85,0.0,0.0,0.0,0.0,204.49,489.76
2,16055443,2,1927,443,2010-12-20,True,1350.0,2000-12-15,2011-01-13,65618.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
3,2994148,2,1919,148,2007-02-15,True,1916.0,1997-01-20,2007-02-21,21685.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
4,4552183,1,1939,183,2010-09-29,True,26270.0,2000-09-22,2010-10-28,27826.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
5,23374431,2,1920,431,2011-12-13,True,1350.0,2001-11-13,2011-12-21,63698.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
6,6726262,1,1933,262,2007-02-23,True,9509.0,1997-02-09,2007-03-23,40443.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,29.29
7,706017,2,1925,17,2008-05-22,True,4693.0,1998-04-18,2008-06-05,2566.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,132.33
8,14778043,1,1921,43,2006-01-31,True,1350.0,1996-01-05,2006-02-23,6185.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00
9,1451179,1,1931,179,2006-09-20,True,1350.0,1996-08-09,2006-09-29,26929.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,308.21


In [75]:
# pt_features.drop(['insom_count_window0', 'insom_count_window1', 'stroke_window0',
#        'stroke_window1',
#        'CHD_heart_failure_and_peripheral_vascular_disease_window0',
#        'CHD_heart_failure_and_peripheral_vascular_disease_window1',
#        'hypertension_window0', 'hypertension_window1', 'diabetes_window0',
#        'diabetes_window1', 'clin_sig_alcohol_use_window0',
#        'clin_sig_alcohol_use_window1', 'mental_illness_non_smi_window0',
#        'mental_illness_non_smi_window1', 'mental_illness_smi_window0',
#        'mental_illness_smi_window1', 'sleep_apnoea_window0',
#        'sleep_apnoea_window1', 'intellectual_disability_window0',
#        'intellectual_disability_window1', 'current_smoker_window0',
#        'current_smoker_window1', 'asthma_window0', 'asthma_window1',
#        'COPD_window0', 'COPD_window1', 'CKD_window0', 'CKD_window1'],axis=1,inplace=True)

In [8]:
pt_features.columns

Index(['patid', 'gender', 'yob', 'pracid', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'insom_count_window0', 'insom_count_window1', 'stroke_window0',
       'stroke_window1',
       'CHD_heart_failure_and_peripheral_vascular_disease_window0',
       'CHD_heart_failure_and_peripheral_vascular_disease_window1',
       'hypertension_window0', 'hypertension_window1', 'diabetes_window0',
       'diabetes_window1', 'clin_sig_alcohol_use_window0',
       'clin_sig_alcohol_use_window1', 'mental_illness_non_smi_window0',
       'mental_illness_non_smi_window1', 'mental_illness_smi_window0',
       'mental_illness_smi_window1', 'sleep_apnoea_window0',
       'sleep_apnoea_window1', 'intellectual_disability_window0',
       'intellectual_disability_window1', 'current_smoker_window0',
       'current_smoker_window1', 'asthma_window0', 'asthma_window1',
       'COPD_window0', 'COPD_window1', 'CKD_window0', 'CKD_window1',
       'mood_stabili

In [9]:
entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [10]:
entries.head(3)

,patid,sysdate,eventdate,medcode,type
0,57001,1997-03-09,1994-10-31,3550,2
1,57001,1997-03-09,1994-10-31,9897,2
2,57001,1997-03-09,1993-01-28,4447,2


In [11]:
windows = get_windows() #[{'end': timedelta(0), 'start': timedelta(3650)}]

In [26]:
# Add insomnia count
pt_features = get_condition_status(pt_features,entries,windows,codelists.insomnia)

medcode_events: 60413, pt_features: 18876


In [27]:
pt_features

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,insomnia_window0,insomnia_window1
0,10665377,1,1920,377,2011-04-07,True,4693.0,2001-04-01,2011-04-14,55943.0,0,0
1,12440330,2,1925,330,2010-03-30,True,4693.0,2000-03-22,2010-04-09,50163.0,0,0
2,16055443,2,1927,443,2010-12-20,True,1350.0,2000-12-15,2011-01-13,65618.0,0,0
3,2994148,2,1919,148,2007-02-15,True,1916.0,1997-01-20,2007-02-21,21685.0,1,1
4,4552183,1,1939,183,2010-09-29,True,26270.0,2000-09-22,2010-10-28,27826.0,0,0
5,23374431,2,1920,431,2011-12-13,True,1350.0,2001-11-13,2011-12-21,63698.0,0,0
6,6726262,1,1933,262,2007-02-23,True,9509.0,1997-02-09,2007-03-23,40443.0,0,1
7,706017,2,1925,17,2008-05-22,True,4693.0,1998-04-18,2008-06-05,2566.0,0,0
8,14778043,1,1921,43,2006-01-31,True,1350.0,1996-01-05,2006-02-23,6185.0,0,0
9,1451179,1,1931,179,2006-09-20,True,1350.0,1996-08-09,2006-09-29,26929.0,0,0


In [28]:
# Add condition status (e.g. diabetes, stroke)
pt_features = get_multiple_condition_statuses(pt_features,entries,windows,codelists.codelist_list_of_lists)

stroke
medcode_events: 48913, pt_features: 18876
CHD_heart_failure_and_peripheral_vascular_disease
medcode_events: 147199, pt_features: 18876
hypertension
medcode_events: 185315, pt_features: 18876
diabetes
medcode_events: 57533, pt_features: 18876
clin_sig_alcohol_use
medcode_events: 2282, pt_features: 18876
mental_illness_non_smi
medcode_events: 140614, pt_features: 18876
mental_illness_smi
medcode_events: 21907, pt_features: 18876
sleep_apnoea
medcode_events: 548, pt_features: 18876
intellectual_disability
medcode_events: 163, pt_features: 18876
current_smoker
medcode_events: 86908, pt_features: 18876
asthma
medcode_events: 41929, pt_features: 18876
COPD
medcode_events: 90029, pt_features: 18876
CKD
medcode_events: 5401, pt_features: 18876


In [12]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [97]:
pt_features = create_pdds(pt_features,prescs,druglists.psychotropic_list_of_lists,get_windows())

In [13]:
all_entries = get_all_entries(medcoded_entries=entries,prescriptions=prescriptions)

medcoded entry length:  56637419
prescription length:  32264418
adding in consultations...
concatenating


In [15]:
all_entries.sample(10)

,eventdate,medcode,patid,prodcode,sysdate,type
85379272,2002-04-15,13354.0,6184196,NaN,2002-04-22,2
74923707,2001-11-12,NaN,2855320,5158.0,2001-11-12,5
115200587,2004-05-05,21.0,791246,NaN,2004-05-05,3
61795495,1996-03-04,NaN,16048181,1.0,2002-04-09,5
115483323,2007-07-10,13.0,6816251,NaN,2007-07-11,3
98522895,2003-04-23,5902.0,928482,NaN,2003-04-23,2
36773415,2006-03-06,NaN,4286519,NaN,2006-03-06,0
46122529,2006-03-09,NaN,3774020,23.0,2006-03-09,5
74799519,2007-03-02,NaN,17987319,5.0,2007-03-02,5
124768599,2012-02-10,13826.0,14188480,NaN,2012-02-10,3


In [18]:
pt_features = get_consultation_count(pt_features,all_entries,get_windows())

In [19]:
pt_features

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,...,sgas_pdds_window0,sgas_pdds_window1,sga_depots_pdds_window0,sga_depots_pdds_window1,fga_depots_pdds_window0,fga_depots_pdds_window1,antidepressants_pdds_window0,antidepressants_pdds_window1,consultation_count_window0,consultation_count_window1
0,10665377,1,1920,377,2011-04-07,True,4693.0,2001-04-01,2011-04-14,55943.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,187,191
1,12440330,2,1925,330,2010-03-30,True,4693.0,2000-03-22,2010-04-09,50163.0,...,0.0,490.85,0.0,0.0,0.0,0.0,204.49,489.76,170,227
2,16055443,2,1927,443,2010-12-20,True,1350.0,2000-12-15,2011-01-13,65618.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,215,124
3,2994148,2,1919,148,2007-02-15,True,1916.0,1997-01-20,2007-02-21,21685.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,58,133
4,4552183,1,1939,183,2010-09-29,True,26270.0,2000-09-22,2010-10-28,27826.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,5,27
5,23374431,2,1920,431,2011-12-13,True,1350.0,2001-11-13,2011-12-21,63698.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,129,208
6,6726262,1,1933,262,2007-02-23,True,9509.0,1997-02-09,2007-03-23,40443.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,29.29,240,251
7,706017,2,1925,17,2008-05-22,True,4693.0,1998-04-18,2008-06-05,2566.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,132.33,215,463
8,14778043,1,1921,43,2006-01-31,True,1350.0,1996-01-05,2006-02-23,6185.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,24,97
9,1451179,1,1931,179,2006-09-20,True,1350.0,1996-08-09,2006-09-29,26929.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,308.21,107,120


In [21]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins.csv',index=False)